Debes crear las siguientes funciones para los endpoints que se consumirán en la API, recuerden que deben tener un decorador por cada una (@app.get(‘/’)).

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

-------------------------------------------------------------------

# Carga de Librerias

In [258]:
import pandas as pd



# Carga de Archivos

In [259]:
df_items = pd.read_csv('items.csv',  index_col=False)
df_reviews = pd.read_csv('reviews_sa.csv', index_col=False)
df_steam = pd.read_csv('steam_games.csv', index_col=False)
df_steam_exploded = pd.read_csv('steam_exploded.csv', index_col=False)

In [260]:
columnas_comunes_1 = df_reviews.columns.intersection(df_steam.columns)

print('Columnas en comun entre reviews  y steam', columnas_comunes_1)

columnas_comunes_2 = df_items.columns.intersection(df_reviews.columns)
print('Columnas en comun entre items  y reviews', columnas_comunes_2)

columnas_comunes_3 = df_items.columns.intersection(df_steam.columns)
print('Columnas en comun entre items  y steam',columnas_comunes_3)



Columnas en comun entre reviews  y steam Index(['user_id', 'item_id'], dtype='object')
Columnas en comun entre items  y reviews Index(['user_id', 'item_id'], dtype='object')
Columnas en comun entre items  y steam Index(['user_id', 'item_id', 'item_name'], dtype='object')


In [136]:
gf = df_steam_exploded[df_steam_exploded['genres'] == 'Simulation']
m = gf.merge(df_items, on=['item_name'], how='left').dropna(subset = ['playtime_forever']).drop_duplicates()

### def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género. Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [261]:
import pandas as pd

def PlayTimeGenre(genero: str):
    # Filtrar juegos del género especificado
    gf = df_steam_exploded[df_steam_exploded['genres'] == genero]

    if gf.empty:
        return {"No se encontraron juegos para el género especificado": None}

    # Realizar un left join para mantener todas las filas de gf y luego eliminar filas con playtime_forever NaN
    m = gf.merge(df_items, on=['item_name'], how='left').dropna(subset=['playtime_forever']).drop_duplicates()

    # Convertir la columna 'release_date' a tipo datetime
    m['release_date'] = pd.to_datetime(m['release_date'], errors='coerce')

    # Agrupar por año de lanzamiento y calcular el tiempo total de juego por año
    year_playtime = m.groupby(m['release_date'].dt.year)['playtime_forever'].sum()

    # Encontrar el año con más horas jugadas
    max_playtime_year = year_playtime.idxmax()

    return {"Año de lanzamiento con más horas jugadas para el género " + genero: int(max_playtime_year)}


#### Probamos la funcion PlayTimeGenre

In [262]:
# Llamar a la función con un género específico, por ejemplo, "Action"
result = PlayTimeGenre("Action")
print(result)

result = PlayTimeGenre("Simulation")
print(result)

{'Año de lanzamiento con más horas jugadas para el género Action': 2012}
{'Año de lanzamiento con más horas jugadas para el género Simulation': 2006}


In [142]:
gf = df_steam_exploded[df_steam_exploded['genres'] == 'Action']
m = gf.merge(df_items, on=['item_name'], how='left').dropna(subset=['playtime_forever']).drop_duplicates()


#### def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

#### Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [265]:
import pandas as pd

def UserForGenre(genero: str):
    # Filtrar juegos del género especificado
    gf = df_steam_exploded[df_steam_exploded['genres'] == genero]

    if gf.empty:
        return {"No se encontraron juegos para el género especificado": None, "Horas jugadas por año": {}}

    # Realizar un left join para mantener todas las filas de gf y luego eliminar filas con playtime_forever NaN
    m = gf.merge(df_items, on=['item_name'], how='left').dropna(subset=['playtime_forever']).drop_duplicates()

    # Convertir la columna 'release_date' a tipo datetime
    m['release_date'] = pd.to_datetime(m['release_date'], errors='coerce')

    # Agrupar por usuario y calcular el tiempo total de juego por usuario
    user_playtime = m.groupby('user_id_y')['playtime_forever'].sum()

    # Encontrar el usuario con más horas jugadas en ese género
    max_playtime_user = user_playtime.idxmax()

    # Filtrar el DataFrame para el usuario con más horas jugadas
    user_max_playtime = m[m['user_id_y'] == max_playtime_user]

    # Calcular la acumulación de horas jugadas por año para ese usuario
    year_playtime = user_max_playtime.groupby(user_max_playtime['release_date'].dt.year)['playtime_forever'].sum()

    # Crear la lista de acumulación de horas jugadas por año en el formato especificado
    horas_por_anio = [{"Año": int(year), "Horas": int(hours)} for year, hours in year_playtime.items()]

    return {"Usuario con más horas jugadas para el género " + genero: max_playtime_user, "Horas jugadas por año": horas_por_anio}



#### Probamos la funcion UserForGenre

In [267]:
result = UserForGenre("Simulation")
print(result)

result = UserForGenre("Strategy")
print(result)

result = UserForGenre("Action")
print(result)

{'Usuario con más horas jugadas para el género Simulation': 'jimmynoe', 'Horas jugadas por año': [{'Año': 2006, 'Horas': 551719}, {'Año': 2009, 'Horas': 15}, {'Año': 2011, 'Horas': 418043}, {'Año': 2012, 'Horas': 1592}, {'Año': 2013, 'Horas': 1751}, {'Año': 2014, 'Horas': 839}, {'Año': 2015, 'Horas': 73812}, {'Año': 2016, 'Horas': 14349}]}
{'Usuario con más horas jugadas para el género Strategy': 'shinomegami', 'Horas jugadas por año': [{'Año': 1993, 'Horas': 22001}, {'Año': 1999, 'Horas': 14}, {'Año': 2001, 'Horas': 2}, {'Año': 2002, 'Horas': 2434}, {'Año': 2003, 'Horas': 531614}, {'Año': 2004, 'Horas': 76}, {'Año': 2007, 'Horas': 68627}, {'Año': 2008, 'Horas': 356}, {'Año': 2009, 'Horas': 8434}, {'Año': 2010, 'Horas': 13733}, {'Año': 2011, 'Horas': 6882}, {'Año': 2012, 'Horas': 79098}, {'Año': 2013, 'Horas': 139630}, {'Año': 2014, 'Horas': 124901}, {'Año': 2015, 'Horas': 3270}, {'Año': 2016, 'Horas': 1379}]}
{'Usuario con más horas jugadas para el género Action': 'shinomegami', 'Hora

##### def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
##### Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [269]:
import pandas as pd

def UsersRecommend(anio: int):
    # Filtrar las revisiones para el año dado y donde recommend sea True y el sentimiento sea positivo o neutral
    reviews_filtered = df_reviews[(df_reviews['posted'].str[-4:] == str(anio)) & \
     (df_reviews['recommend'] == True) & (df_reviews['sentiment_analysis']> 0)]#.isin([1, 2]))]

    # Realizar un merge para obtener los nombres de los juegos
    merged_reviews = reviews_filtered.merge(df_steam[['item_id', 'item_name']], on='item_id', how='left')


    # Agrupar por item_name y contar las revisiones para cada juego
    game_counts = merged_reviews.groupby('item_name')['recommend'].count().reset_index()

    # Ordenar los juegos por la cantidad de revisiones en orden descendente
    top_games = game_counts.sort_values(by='recommend', ascending=False)

    # Tomar los 3 juegos principales
    top_3_games = top_games.head(3)

    # Crear la lista de retorno con el formato especificado
    result = [{"Puesto {}: ".format(i + 1): game} for i, game in enumerate(top_3_games['item_name'])]

    return result


##### Probamos la funcion UsersRecommend

In [152]:
for i in range(2009, 2017):
  print(f'Año {i} : {UsersRecommend(i)}')

Año 2009 : []
Año 2010 : [{'Puesto 1: ': 'Team Fortress 2'}, {'Puesto 2: ': 'Killing Floor'}, {'Puesto 3: ': 'Alien Swarm'}]
Año 2011 : [{'Puesto 1: ': 'Team Fortress 2'}, {'Puesto 2: ': 'Portal 2'}, {'Puesto 3: ': 'Terraria'}]
Año 2012 : [{'Puesto 1: ': 'Team Fortress 2'}, {'Puesto 2: ': 'Terraria'}, {'Puesto 3: ': "Garry's Mod"}]


Año 2013 : [{'Puesto 1: ': 'Team Fortress 2'}, {'Puesto 2: ': "Garry's Mod"}, {'Puesto 3: ': 'Left 4 Dead 2'}]
Año 2014 : [{'Puesto 1: ': 'Team Fortress 2'}, {'Puesto 2: ': 'Counter-Strike: Global Offensive'}, {'Puesto 3: ': "Garry's Mod"}]
Año 2015 : [{'Puesto 1: ': 'Counter-Strike: Global Offensive'}, {'Puesto 2: ': 'Team Fortress 2'}, {'Puesto 3: ': "Garry's Mod"}]
Año 2016 : []


##### def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos) Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [153]:
def UsersNotRecommend(anio: int):
    # Filtrar las revisiones para el año dado, donde recommend sea False y el sentimiento sea negativo
    reviews_filtered = df_reviews[(df_reviews['posted'].str[-4:] == str(anio)) & \
                                  (df_reviews['recommend'] == False) & (df_reviews['sentiment_analysis'] == 0)]

    # Realizar un merge para obtener los nombres de los juegos
    merged_reviews = reviews_filtered.merge(df_steam[['item_id', 'item_name']], on='item_id', how='left')

    # Agrupar por item_name y contar las revisiones para cada juego
    game_counts = merged_reviews.groupby('item_name')['recommend'].count().reset_index()

    # Ordenar los juegos por la cantidad de revisiones en orden descendente
    bottom_games = game_counts.sort_values(by='recommend', ascending=True)

    # Tomar los 3 juegos menos recomendados
    bottom_3_games = bottom_games.head(3)

    # Crear la lista de retorno con el formato especificado
    result = [{"Puesto {}: ".format(i + 1): game} for i, game in enumerate(bottom_3_games['item_name'])]

    return result


##### Probamos la funcion  UsersNotRecommend

In [273]:
for i in range(2010, 2017):
    # do something
  print(f'Año {i} : {UsersNotRecommend(i)}')

Año 2010 : []
Año 2011 : [{'Puesto 1: ': 'Men of War: Vietnam'}, {'Puesto 2: ': 'Team Fortress 2'}, {'Puesto 3: ': 'And Yet It Moves'}]
Año 2012 : [{'Puesto 1: ': 'Call of Duty®: Black Ops II'}, {'Puesto 2: ': 'Red Faction®: Armageddon™'}, {'Puesto 3: ': 'Resident Evil: Operation Raccoon City'}]
Año 2013 : [{'Puesto 1: ': 'A.V.A. Alliance of Valiant Arms™'}, {'Puesto 2: ': 'Warframe'}, {'Puesto 3: ': 'Nether: Resurrected'}]
Año 2014 : [{'Puesto 1: ': 'Mabinogi'}, {'Puesto 2: ': 'NEOTOKYO'}, {'Puesto 3: ': 'Napoleon: Total War™'}]
Año 2015 : [{'Puesto 1: ': '100% Orange Juice'}, {'Puesto 2: ': 'Morphopolis'}, {'Puesto 3: ': 'Miscreated'}]
Año 2016 : []


##### def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento. Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}


In [274]:
def sentiment_analysis(anio: int):
    # Filtrar las revisiones para el año de lanzamiento dado
    reviews_filtered = df_steam[df_steam['release_date'].str[:4] == str(anio)]

    # Realizar un merge entre las revisiones filtradas y df_reviews para obtener el análisis de sentimiento
    merged_reviews = reviews_filtered.merge(df_reviews[['item_id', 'sentiment_analysis']], on='item_id', how='left')

    # Contar las revisiones con análisis de sentimiento y agrupar por categoría de sentimiento
    sentiment_counts = merged_reviews['sentiment_analysis'].value_counts()

    # Crear un diccionario con las categorías de sentimiento y sus conteos
    result = {}
    for sentiment_category, count in sentiment_counts.items():
        result[str(sentiment_category)] = count

    # Verificar si no hay reseñas para el año y establecer todos los valores en 0
    if not result:
        result = {'0.0': 0, '1.0': 0, '2.0': 0}

    new_result = {'Negative': result.get('0.0', 0), 'Neutral': result.get('1.0', 0), 'Positive': result.get('2.0', 0)}

    return new_result

In [275]:
for i in range(1995, 2020):
  print(f'Año {i} : {sentiment_analysis(i)}')

Año 1995 : {'Negative': 5, 'Neutral': 0, 'Positive': 10}
Año 1996 : {'Negative': 0, 'Neutral': 2, 'Positive': 1}
Año 1997 : {'Negative': 6, 'Neutral': 6, 'Positive': 29}
Año 1998 : {'Negative': 16, 'Neutral': 12, 'Positive': 54}
Año 1999 : {'Negative': 6, 'Neutral': 11, 'Positive': 23}
Año 2000 : {'Negative': 8, 'Neutral': 25, 'Positive': 54}
Año 2001 : {'Negative': 8, 'Neutral': 2, 'Positive': 17}
Año 2002 : {'Negative': 6, 'Neutral': 4, 'Positive': 27}
Año 2003 : {'Negative': 41, 'Neutral': 42, 'Positive': 103}
Año 2004 : {'Negative': 62, 'Neutral': 115, 'Positive': 324}
Año 2005 : {'Negative': 34, 'Neutral': 51, 'Positive': 169}
Año 2006 : {'Negative': 211, 'Neutral': 417, 'Positive': 1207}
Año 2007 : {'Negative': 348, 'Neutral': 934, 'Positive': 2527}
Año 2008 : {'Negative': 48, 'Neutral': 71, 'Positive': 275}
Año 2009 : {'Negative': 294, 'Neutral': 414, 'Positive': 1079}
Año 2010 : {'Negative': 278, 'Neutral': 314, 'Positive': 1193}
Año 2011 : {'Negative': 413, 'Neutral': 646, 'Po

Modelo de aprendizaje automático:

Una vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un sistema de recomendación. Para ello, te ofrecen dos propuestas de trabajo: En la primera, el modelo deberá tener una relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicar la similitud del coseno. La otra propuesta para el sistema de recomendación debe aplicar el filtro user-item, esto es tomar un usuario, se encuentran usuarios similares y se recomiendan ítems que a esos usuarios similares les gustaron. En este caso el input es un usuario y el output es una lista de juegos que se le recomienda a ese usuario, en general se explican como “A usuarios que son similares a tí también les gustó…”. Deben crear al menos uno de los dos sistemas de recomendación (Si se atreven a tomar el desafío, para mostrar su capacidad al equipo, ¡pueden hacer ambos!). Tu líder pide que el modelo derive obligatoriamente en un GET/POST en la API símil al siguiente formato:

Si es un sistema de recomendación item-item:

def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.
Si es un sistema de recomendación user-item:

def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

Dataframe que voy a usar : reviews , utilizaré las recomendaciones , de usuarios que recomendaron los mismos juegos para recomendar juegos , en base al analisis de sentimiento de la review del usuario

In [277]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48498 entries, 0 to 48497
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             48498 non-null  object
 1   posted              48498 non-null  object
 2   item_id             48498 non-null  int64 
 3   recommend           48498 non-null  bool  
 4   sentiment_analysis  48498 non-null  int64 
dtypes: bool(1), int64(2), object(2)
memory usage: 1.5+ MB


In [283]:
df_reviews.head()

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,05-11-2011,1250,True,2
1,76561197970982479,15-07-2011,22200,True,2
2,76561197970982479,21-04-2011,43110,True,2
3,js41637,24-06-2014,251610,True,2
4,js41637,08-09-2013,227300,True,2


Por una cuestion de recursos computacionales , voy a utilizar solo los primeros 10000 datos

Los voy a mezclar , y luego crear una tabla pivote con python y luego calcular la similitud del coseno , que mide la distancia angular entre vectores

In [293]:

df_reviews_shuffled = df_reviews.head(10000)
df_reviews_shuffled = df_reviews_shuffled.sample(frac=1, random_state=42) 

from sklearn.metrics.pairwise import cosine_similarity

# Crear una matriz de usuarios como características y juegos como filas
user_item_matrix = df_reviews_shuffled.pivot_table(index='user_id', columns='item_id', values='sentiment_analysis').fillna(0)

# Calcular la similitud entre usuarios usando la similitud del coseno
user_similarity = cosine_similarity(user_item_matrix)

df_reviews_shuffled.head()

In [294]:
def recomendacion_usuario(user_id):
    # Obtener la fila correspondiente al usuario ingresado
    user_vector = user_item_matrix.loc[user_id].values.reshape(1, -1)
    
    # Calcular la similitud entre el usuario ingresado y todos los demás usuarios
    similarities = cosine_similarity(user_vector, user_item_matrix)
    
    # Obtener los juegos que los usuarios similares a 'user_id' han disfrutado
    user_reviews = user_item_matrix.loc[user_id]
    similar_users = df_reviews['user_id'][similarities.argsort()[0][-6:-1]]
    recommended_items = user_item_matrix.loc[similar_users].mean(axis=0).sort_values(ascending=False)
    
    # Filtrar los juegos que el usuario ya ha jugado
    recommended_items = recommended_items[recommended_items.index.isin(user_reviews[user_reviews == 0].index)]
    
    return recommended_items.index.tolist()[:5]

In [296]:
df_reviews_shuffled.head(3)

,user_id,posted,item_id,recommend,sentiment_analysis
6252,barbas1,26-10-2013,4000,True,2
4684,76561198056212336,26-11-2013,236850,True,2
1731,76561198103524411,24-07-2014,200510,True,2


In [297]:
df_reviews_shuffled.tail()

,user_id,posted,item_id,recommend,sentiment_analysis
5734,UniIsBestWaifu,24-09-2015,333980,True,2
5191,Z1_M4N,26-07-2012,105600,True,2
5390,76561198080889775,22-05-2014,34330,True,1
860,bobseagull,07-05-2014,49520,True,0
7270,76561197996426485,17-10-2014,241930,True,2


In [298]:
recomendacion_usuario('barbas1')

[440, 10680, 239220, 202170, 236370]

In [299]:
df_reviews_shuffled[df_reviews_shuffled['user_id'] == 'barbas1']

,user_id,posted,item_id,recommend,sentiment_analysis
6252,barbas1,26-10-2013,4000,True,2


In [301]:
df_steam[df_steam['item_id'] == 4000]

,genres,item_name,release_date,item_id,user_id
46,['Indie' 'Simulation'],Garry's Mod,2006-11-29,4000,NaN


In [302]:
# [440, 10680, 239220, 202170, 236370]

In [303]:
df_steam[df_steam['item_id'] == 440]

,genres,item_name,release_date,item_id,user_id
28584,['Action' 'Free to Play'],Team Fortress 2,2007-10-10,440,NaN


In [304]:
df_steam[df_steam['item_id'] == 10680]

,genres,item_name,release_date,item_id,user_id
28249,['Action'],Aliens vs. Predator™,2010-02-16,10680,NaN


In [233]:
df_steam[df_steam['item_id'] == 236370]

,genres,item_name,release_date,item_id,user_id
26864,['Action' 'Indie' 'RPG' 'Simulation' 'Early Ac...,Interstellar Marines,2013-07-02,236370,NaN


In [305]:
df_items[df_items['item_id'] == 239220]

,user_id,item_id,item_name,playtime_forever


In [306]:
df_steam[df_steam['item_id'] == 239220]

,genres,item_name,release_date,item_id,user_id


In [307]:
df_items[df_items['item_id'] == 202170]

,user_id,item_id,item_name,playtime_forever


In [236]:
df_steam[df_steam['item_id'] == 202170]

,genres,item_name,release_date,item_id,user_id


In [308]:
recomendacion_usuario('UniIsBestWaifu')

[265630, 240, 8870, 227700, 570]

In [242]:
df_reviews[df_reviews['user_id'] == 'UniIsBestWaifu']

,user_id,posted,item_id,recommend,sentiment_analysis
5733,UniIsBestWaifu,07-09-2015,17730,False,0
5734,UniIsBestWaifu,24-09-2015,333980,True,2


In [249]:
df_steam[df_steam['item_id'] == 17730]

,genres,item_name,release_date,item_id,user_id
28398,['Sports' 'Action'],Smashball,Jun 2009,17730,NaN


In [247]:
df_steam[df_steam['item_id'] == 333980]

,genres,item_name,release_date,item_id,user_id
3644,['Action' 'Adventure' 'Casual' 'RPG'],AKIBA'S TRIP: Undead ＆ Undressed,2015-05-26,333980,NaN


In [248]:
df_reviews[df_reviews['user_id'] == 'UniIsBestWaifu']

,user_id,posted,item_id,recommend,sentiment_analysis
5733,UniIsBestWaifu,07-09-2015,17730,False,0
5734,UniIsBestWaifu,24-09-2015,333980,True,2


Vemos que no recomendo el primero pero si el segundo

In [309]:
recomendacion_usuario('UniIsBestWaifu')

[265630, 240, 8870, 227700, 570]

In [250]:
df_steam[df_steam['item_id'] == 240]

,genres,item_name,release_date,item_id,user_id
28725,['Action'],Counter-Strike: Source,2004-11-01,240,NaN


In [252]:
df_steam[df_steam['item_id'] == 8870]

,genres,item_name,release_date,item_id,user_id
27034,['Action'],BioShock Infinite,2013-03-25,8870,NaN


In [254]:
df_items[df_items['item_id'] == 570]

,user_id,item_id,item_name,playtime_forever


In [310]:
df_steam[df_steam['item_id'] == 570]

,genres,item_name,release_date,item_id,user_id


In [255]:
df_items[df_items['item_id'] == 265630]

,user_id,item_id,item_name,playtime_forever


In [256]:
df_steam[df_steam['item_id'] == 265630]

,genres,item_name,release_date,item_id,user_id
2095,['Action' 'Free to Play' 'Indie'],Fistful of Frags,2014-05-09,265630,NaN


In [257]:
df_steam[df_steam['item_id'] == 227700]

,genres,item_name,release_date,item_id,user_id
